# OPS-SAT case starter-kit notebook

ESA's [Kelvins](https://kelvins.esa.int) competition "[the OPS-SAT case](https://kelvins.esa.int/opssat/home/)" is a novel data-centric challenge that asks you to work with the raw data of a satellite and very few provided labels to find the best parameters for a given machine learning model. Compared to previous competitions on Kelvins (like the [Pose Estimation](https://kelvins.esa.int/pose-estimation-2021/) or the [Proba-V Super-resolution challenge](https://kelvins.esa.int/proba-v-super-resolution/)) where the test-set is provided and the infered results are submitted, for the OPS-SAT case, we will run inference on the Kelvins server directly! This notebooks contains examples on how you can load your data and train an **EfficientNetLite0** model by only using the 80-labeled images provided. Therefore, the directory `images`, containing unlabeld patches and included in the training dataset is not used for this notebook. However, competitors are encouraged to use these patches to improve the model accuracy.

# 1. Module imports

If you do not have a GPU, uncomment and run the next commands.


In [146]:
import sys
sys.version

'3.9.16 (main, Mar  8 2023, 14:00:05) \n[GCC 11.2.0]'

In [147]:
! python --version

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Python 3.9.16


In [148]:
! python --version

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Python 3.9.16


In [149]:
! which python

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
/home/ramez/miniconda3/envs/thesis/bin/python


In [150]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"

Other imports.

In [151]:
# !nvidia-smi

In [152]:
# !conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [153]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/

In [154]:
# !mkdir -p $CONDA_PREFIX/etc/conda/activate.d

In [155]:
# !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/' > $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh

In [156]:
# !pip install tensorflow==2.7.0

In [157]:
import tensorflow as tf

In [158]:
# !python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [159]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [160]:
from tensorflow import keras

In [161]:
import numpy as np

In [162]:
# !pip install scikit-learn==1.0.2

In [163]:
from sklearn.metrics import cohen_kappa_score

In [164]:
import site
print(site.getsitepackages())


['/home/ramez/miniconda3/envs/thesis/lib/python3.9/site-packages']


In [165]:
from efficientnet_lite import EfficientNetLiteB0

In [166]:
from sklearn.model_selection import KFold

In [167]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [168]:
from tensorflow.keras.callbacks import TensorBoard

# 2. Utility Functions

You can use this function to load your training data.

In [169]:
def get_images_from_path(dataset_path):
    """ Get images from path and normalize them applying channel-level normalization. """

    # loading all images in one large batch
    tf_eval_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], shuffle=False, 
                                                               batch_size=100000)

    # extract images and targets
    for tf_eval_images, tf_eval_targets in tf_eval_data:
        break

    return tf.convert_to_tensor(tf_eval_images), tf_eval_targets

# 3. Loading the model

The network architecture used for OPS-SAT is **EfficientNetLite0**. We would like to thank Sebastian for making a Keras implementation of EfficientNetLite publicly available under the Apache 2.0 License: https://github.com/sebastian-sz/efficientnet-lite-keras. Our Version of this code has been modified to better fit our purposes. For example, we removed the ReLU "stem_activation" to better match a related efficientnet pytorch implementation. In any way, **you have to use the model architecture that we provide in our [starter-kit](https://gitlab.com/EuropeanSpaceAgency/the_opssat_case_starter_kit).**

In [170]:
input_shape = (200, 200, 3)   # input_shape is (height, width, number of channels) for images
num_classes = 8

In [171]:
model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
model.summary()

block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same
Model: "efficientnetlite0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 201, 201, 3)  0           ['input_21[0][0]']               
                                               

In [172]:
model = EfficientNetLiteB0(classes=num_classes, weights='imagenet', input_shape=input_shape, classifier_activation=None, include_top = False)
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output_layer = Dense(8, activation='softmax')(x)
model = Model(inputs=model.input, outputs=output_layer)
model.summary()

block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 201, 201, 3)  0           ['input_22[0][0]']               
                                                         

# 4. Loading data

Uncomment next line and adjust with the path of your dataset. 

In [173]:
# dataset_path_train="/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/images_copy_processed/"
dataset_path_train_val = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_train/"

In [174]:
dataset_path_test = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_test/"

In this notebook, classical supervised learning is used. Therefore, remember to remove the subdirectory `images` containing unlabeled patches before loading the dataset to perform training correctly.

In [175]:
#Loading dataset
x_train_val, y_train_val = get_images_from_path(dataset_path_train_val)

Found 227 files belonging to 8 classes.


In [176]:
#Loading dataset
x_test, y_test = get_images_from_path(dataset_path_test)

Found 40 files belonging to 8 classes.


# 5. Model training

We provide now an example on how you can train your model by using standard supervised learning. Training loss (`SparseCategoricalCrossentropy`) and `Accuracy` are shown for simplicity and for an easier interpretation of the training outcome, despite your submission will be evaluated by using the metric **1 - Cohen's kappa** [metric](https://en.wikipedia.org/wiki/Cohen's_kappa). For more information on scoring, please refer to [Scoring](https://kelvins.esa.int/opssat/scoring/).

With this model and the dataset provided, please do your best!

In [177]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [178]:
# load data, data augmentation, training, overfitting, transfer-learning etc.
# history=model.fit(x_train, y_train, epochs=20, verbose=1, batch_size=8)

Calculating the **1 - Cohen's kappa** score of the trained model on the trained dataset.

In [179]:
# predictions = np.zeros(len(y_test), dtype=np.int8)
# # inference loop
# for e, (image, target) in enumerate(zip(x_test, y_test)):
#     image = np.expand_dims(np.array(image), axis=0)
#     output = model.predict(image)
#     predictions[e] = np.squeeze(output).argmax()
# #Keras model score
# score_keras = 1 - cohen_kappa_score(y_test.numpy(), predictions)
# print("Score:",score_keras)

In [180]:
k = 5
kf = KFold(n_splits=k, shuffle=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [181]:
tensorboard_callback = TensorBoard(log_dir='./logs')

In [182]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(x_train_val, y_train_val, validation_data=(x_test, y_test), epochs= 100, verbose=1, batch_size=8, 
                        callbacks=[early_stopping, tensorboard_callback])

/home/ramez/miniconda3/envs/thesis/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/100


/home/ramez/miniconda3/envs/thesis/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


29/29 [==============================] - 7s 137ms/step - loss: 1.4446 - sparse_categorical_accuracy: 0.5110 - val_loss: 2.0929 - val_sparse_categorical_accuracy: 0.3250
Epoch 2/100
29/29 [==============================] - 3s 110ms/step - loss: 0.8837 - sparse_categorical_accuracy: 0.7093 - val_loss: 2.9152 - val_sparse_categorical_accuracy: 0.3250
Epoch 3/100
29/29 [==============================] - 3s 111ms/step - loss: 0.7845 - sparse_categorical_accuracy: 0.7445 - val_loss: 2.3256 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/100
29/29 [==============================] - 3s 110ms/step - loss: 0.6347 - sparse_categorical_accuracy: 0.7489 - val_loss: 2.5923 - val_sparse_categorical_accuracy: 0.4000
Epoch 5/100
29/29 [==============================] - 3s 108ms/step - loss: 0.6763 - sparse_categorical_accuracy: 0.7665 - val_loss: 1.7080 - val_sparse_categorical_accuracy: 0.5000
Epoch 6/100
29/29 [==============================] - 3s 108ms/step - loss: 0.5683 - sparse_categorical_accu

In [183]:
# # Train and evaluate the model using K-fold cross-validation
# scores = []
# for train_idx, val_idx in kf.split(x_train_val):
#     X_train = tf.gather(x_train_val, train_idx)
#     y_train = tf.gather(y_train_val, train_idx)
#     X_val = tf.gather(x_train_val, val_idx)
#     y_val = tf.gather(y_train_val, val_idx)
#     history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= 100, verbose=1, batch_size=8, 
#                         callbacks=[early_stopping, tensorboard_callback])

#     score = model.evaluate(X_val, y_val)
#     scores.append(score[1])

In [184]:
# Print the mean validation accuracy
print('Validation accuracy mean: {:.2f} (std {:.2f})'.format(np.mean(scores), np.std(scores)))

Validation accuracy mean: 0.93 (std 0.06)


In [185]:
# Print the mean validation accuracy
print('Validation accuracy mean: {:.2f} (std {:.2f})'.format(np.mean(scores), np.std(scores)))

Validation accuracy mean: 0.93 (std 0.06)


In [186]:
!tensorboard --logdir=./logs

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
predictions = np.zeros(len(y_test), dtype=np.int8)
# inference loop
for e, (image, target) in enumerate(zip(x_test, y_test)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()
#Keras model score
score_keras = 1 - cohen_kappa_score(y_test.numpy(), predictions)
print("Score:",score_keras)

# 6. Saving and loading trained model

The trained model can be now saved by using HDF5-format that is the only accepted for submission. The name `test.h5` will be used.

In [ ]:
#Saving model
# model.save_weights('test.h5')
model.save_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/submit_19.h5')

The trained model can be also loaded for further testing. 

In [ ]:
model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# model = EfficientNetLiteB0(include_top=False, classes=num_classes, weights='imagenet', input_shape=input_shape, classifier_activation=None)
model.load_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/model_patterns_20epochs.h5')

The model will be now compiled and tested again. You should get the same score as before saving and loading. 

In [ ]:
#Model shall be compiled before testing.
model.compile()

#Creating empty predictions
predictions = np.zeros(len(y_train), dtype=np.int8)

# inference loop
for e, (image, target) in enumerate(zip(x_train, y_train)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()

#Keras model score
score_keras = 1 - cohen_kappa_score(y_train.numpy(), predictions)
print("Score:",score_keras)